In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import seaborn as sns
from skimage.transform import resize
from skimage.measure import block_reduce
import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, Flatten, Dropout, Dense, concatenate
import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc, accuracy_score
from numpy import where
import tensorflow_addons as tfa
from tensorflow.keras import Model, Input

In [ ]:
!git clone https://github.com/rininobaron/Kaggle_Melanoma_2020.git

In [ ]:
train=pd.read_csv('./Kaggle_Melanoma_2020/train_sample.csv')
test=pd.read_csv('./Kaggle_Melanoma_2020/test_sample.csv')

In [ ]:
def get_images_to_np(df=pd.read_csv('./Kaggle_Melanoma_2020/test_sample.csv'), submission=False):
    #Parameters
    #df: Pandas DataFrame with the names of images
    #submission:boolea, if True df is equal to pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
    #
    #Returns
    #X: numpy array with images
    
    if submission:
        df=pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
        path_root='../input/siim-isic-melanoma-classification/jpeg/test/'
    
    path_root='../input/siim-isic-melanoma-classification/jpeg/train/'
    
    #Reference image to standardize reslution
    img = mpimg.imread(path_root+'ISIC_0015719.jpg')
    shape1=img.shape[0]
    shape2=img.shape[1]
    
    #Our image is very large then we propose reduce the resolution iimage by a factor_reduce
    factor_reduce=25
    X = np.zeros((df['image_name'].count(),int(shape1/factor_reduce),int(shape2/factor_reduce),img.shape[2]))
    
    count=0
    for i in df['image_name']:
        
        print('Processing image in row: '+str(count))
        
        #Getting image
        img = mpimg.imread(path_root+i+'.jpg')
        print(type(np.max(img)))
        
        #sns.distplot(img[:,:,0],kde=False, rug=True)
        #a=plt.hist(img[:,:,2].ravel(), bins=256, range=(0.0, 255.0), fc='k', ec='k')
        #plt.show(a)
        
        #Ensure standardize reslution
        if shape1!=img.shape[0] or shape2!=img.shape[1]:
            img=resize(img,(shape1,shape2))
            #Convert the image to a 0-255 scale.
            img=255*img
            #Convert to integer data type pixels.
            img = img.astype(np.uint8)
        
        #sns.distplot(img[:,:,0],kde=False, rug=True)
        #b=plt.hist(img[:,:,2].ravel(), bins=256, range=(0.0, 255.0), fc='k', ec='k')
        #plt.show(b)
        
        #Force image to uint8
        #img=img_as_ubyte(img)
        
        #Display important values to track function behavior
        print(np.max(img))
        print(type(np.max(img)))
        
        #Display the first four images in train
        if count<=3:
            print('Image name is: '+i)
            temp=plt.imshow(img, vmin=0,vmax=255)
            plt.show(temp)
        
        #But the image is very large yet
        #Therefore we propose to reduce it using from skimage.measure.block_reduce
        #and np.mean
        img = block_reduce(img, block_size=(factor_reduce, factor_reduce, 1), func=np.mean)
        
        #Convert to integer data type pixels.
        img = img.astype(np.uint8)
        
        #Saving img in X array
        
        X[count,:,:,:]=img
        
        #Display the first four images in train
        if count<=3:
            print('Image name reduce by factor '+str(factor_reduce)+' is: '+i)
            temp=plt.imshow(img, vmin=0,vmax=255)
            plt.show(temp)
        
        count+=1
        
        
    return X

In [ ]:
X_test=get_images_to_np(df=test, submission=False)

In [ ]:
X_test.shape

In [ ]:
X_train=get_images_to_np(df=train, submission=False)

In [ ]:
X_train.shape

# Setting true outputs

In [ ]:
Y_train=train['target'].to_numpy()
Y_test=test['target'].to_numpy()

# Setting weights per class to try to fix unbalanced class problem

In [ ]:
w_p=np.sum(Y_train==0)/Y_train.shape[0]
w_n=np.sum(Y_train==1)/Y_train.shape[0]

In [ ]:
#Creating dictionary
class_weights = { 0 : w_n , 1 : w_p }

# Defining Important parameters for training

In [ ]:
# parameters
EPOCHS = 200
INIT_LR = 1e-4
BS = X_train.shape[0]
CLASSES = 1 #Very important number
#norm_size = 28

opt=tf.keras.optimizers.Nadam(lr=INIT_LR, decay=INIT_LR / EPOCHS, clipvalue=5) #Setting clipvalue to avoid exploding gradientes
#opt=tfa.optimizers.LAMB(learning_rate=INIT_LR, decay= INIT_LR / EPOCHS,clipvalue=5)

# start to train model
print('start to train model')

#Setting Metrics
METRICS = [
  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
  tf.keras.metrics.Precision(name='precision'),
  tf.keras.metrics.Recall(name='recall'),
  tf.keras.metrics.AUC(name='AUC')
]

# Output Model

In [ ]:
# Global Constants
WEIGHT_DECAY=0.005
LRN2D_NORM=True
DATA_FORMAT='channels_last' # Theano:'channels_first' Tensorflow:'channels_last'
USE_BN=True

def conv2D_lrn2d(x,filters,kernel_size,strides=(1,1),padding='same',dilation_rate=(1,1),activation='relu',
                 use_bias=True,kernel_initializer='he_normal',bias_initializer='zeros',
                 kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,
                 kernel_constraint=None,bias_constraint=None,lrn2d_norm=LRN2D_NORM,weight_decay=WEIGHT_DECAY):
    #l2 normalization
    if weight_decay:
        kernel_regularizer=regularizers.l2(weight_decay)
        bias_regularizer=regularizers.l2(weight_decay)
    else:
        kernel_regularizer=None
        bias_regularizer=None
    x=Conv2D(filters=filters,kernel_size=kernel_size,strides=strides,padding=padding,dilation_rate=dilation_rate,
             activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
             bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
             activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(x)
    if lrn2d_norm:
        #batch normalization
        x=BatchNormalization()(x)

    return x

def inception_module(x,params,concat_axis,padding='same',dilation_rate=(1,1),activation='relu',
                     use_bias=True,kernel_initializer='he_normal',bias_initializer='zeros',
                     kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,
                     bias_constraint=None,lrn2d_norm=LRN2D_NORM,weight_decay=None):
    (branch1,branch2,branch3,branch4)=params
    if weight_decay:
        kernel_regularizer=regularizers.l2(weight_decay)
        bias_regularizer=regularizers.l2(weight_decay)
    else:
        kernel_regularizer=None
        bias_regularizer=None
    #1x1
    pathway1=Conv2D(filters=branch1[0],kernel_size=(1,1),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(x)
    #1x1->3x3
    pathway2=Conv2D(filters=branch2[0],kernel_size=(1,1),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(x)
    pathway2=Conv2D(filters=branch2[1],kernel_size=(3,3),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(pathway2)
    #1x1->5x5
    pathway3=Conv2D(filters=branch3[0],kernel_size=(1,1),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(x)
    pathway3=Conv2D(filters=branch3[1],kernel_size=(5,5),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(pathway3)
    #3x3->1x1
    pathway4=MaxPooling2D(pool_size=(3,3),strides=1,padding=padding,data_format=DATA_FORMAT)(x)
    pathway4=Conv2D(filters=branch4[0],kernel_size=(1,1),strides=1,padding=padding,dilation_rate=dilation_rate,
                    activation=activation,use_bias=use_bias,kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer,kernel_regularizer=kernel_regularizer,bias_regularizer=bias_regularizer,
                    activity_regularizer=activity_regularizer,kernel_constraint=kernel_constraint,bias_constraint=bias_constraint)(pathway4)

    return concatenate([pathway1,pathway2,pathway3,pathway4],axis=concat_axis)

class inceptionV3:
    @staticmethod
    def build(width, height, depth, NB_CLASS):
        INP_SHAPE = (height, width, depth)
        img_input = Input(shape=INP_SHAPE)
        CONCAT_AXIS = 3
        
        x = inception_module(img_input, params=[(64,), (96, 128), (16, 32), (32,)], concat_axis=CONCAT_AXIS)  # 3a

        # Create a Keras Model
        model = Model(inputs=img_input, outputs=[x])
        model.summary()
        # Save a PNG of the Model Build
        #plot_model(model, to_file='../imgs/GoogLeNet.png')
        # return the constructed network architecture
        return model

# Setting baseline model

In [ ]:
rnet = InceptionV3(weights='imagenet', include_top=False ,input_shape=[X_train.shape[1],X_train.shape[2],X_train.shape[3]])    
rnet.trainable=False
rnet.summary()

In [ ]:
new_model = tf.keras.Sequential([rnet])
new_model.summary()

In [ ]:
model2 = inceptionV3.build(width=3, height=6, depth=2048, NB_CLASS=100) 
model2.summary()

# Merging models

In [ ]:
new_model.add(model2)
new_model.add(tf.keras.layers.GlobalAveragePooling2D())
new_model.add(Dropout(0.4)) #In order to avoid overfitting
#To avoid overfitting kernel regularizer was setting with two parameters
new_model.add(Dense(units=CLASSES, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.1)))

In [ ]:
new_model.summary()

# Baseline Model Architecture

In [ ]:
print("BASELINE MODEL ARCHITECTURE")
rnet.summary()
print()

# Final Model Architecture

In [ ]:
print("FINAL MODEL TO TRAINING AND TESTING")
new_model.summary()

# Compiling Model

In [ ]:
new_model.compile(
    optimizer=opt,
    loss = 'binary_crossentropy',
    metrics=METRICS
)

# Use Data Augmentation

In [ ]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                             horizontal_flip=True, fill_mode="nearest")

# Training Model

In [ ]:
history_1 = new_model.fit(aug.flow(X_train, Y_train, batch_size=BS),
                            steps_per_epoch=len(X_train) // BS, validation_data=aug.flow(X_test, Y_test),
                            epochs=EPOCHS, verbose=2, class_weight=class_weights)

# Setting important function to visualize metrics

In [ ]:
#Setting function to display confusion matrix
def display_confusion_matrix(cmat, score, precision, recall):   ###1
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(CLASSES+1))
    ax.set_xticklabels(["0","1"], fontdict={'fontsize': 34})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(CLASSES+1))
    ax.set_yticklabels(["0","1"], fontdict={'fontsize': 34})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(1.25, -0.25, titlestring, fontdict={'fontsize': 22, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#000000'})
    plt.title("Confusion Matrix", fontdict={'fontsize': 34})
    plt.savefig('Matriz_de_Confusion.png')
    plt.show()
    

#Setting function to display training curves
def display_training_curves(training, validation, title, subplot):   ####2
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])
    plt.savefig(str(title)+'.png')

#Setting function to display ROC curve
def plot_roc(name, labels, predictions):
    fp, tp, _ = roc_curve(labels, predictions)
    auc1 = auc(fp, tp)
    lw=2

    plt.plot(100*fp, 100*tp, linewidth=2, lw=lw, color='darkorange', label='ROC curve (area = %0.2f)' % auc1)
    plt.plot([0, 100], [0, 100], color='navy', lw=lw, linestyle='--',label='Random')
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.xlim([-5,105])
    plt.ylim([-5,105])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')
    plt.title(name)
    plt.legend(loc="lower right")
    plt.savefig('ROC.png')


# Plotting Validation Metrics

In [ ]:
#Setting class predict vectors
y_train_pred_labeled=(new_model.predict(X_train) > 0.5).astype("int32")
y_test_pred_labeled=(new_model.predict(X_test) > 0.5).astype("int32")
print()
print('y_test_pred_labeled:')
print(y_test_pred_labeled)
print()

#Setting validation metrics
print('Prediction probabilities: '+str(new_model.predict(X_test)))
print()
print()
#METRICS FROM FINAL WEIGHTS MODEL
cmat = confusion_matrix(Y_test, y_test_pred_labeled)
score = f1_score(Y_test, y_test_pred_labeled)
precision = precision_score(Y_test, y_test_pred_labeled)
recall = recall_score(Y_test, y_test_pred_labeled)

#Display training curves (loss & accuracy)
display_training_curves(history_1.history['loss'], history_1.history['val_loss'], 'loss', 211)
display_training_curves(history_1.history['accuracy'], history_1.history['val_accuracy'], 'accuracy', 212)

#Display training curves (loss & accuracy)
precision_history1=np.array(history_1.history['precision'])
precision_history2=np.array(history_1.history['val_precision'])

recall_history1=np.array(history_1.history['recall'])
recall_history2=np.array(history_1.history['val_recall'])

f1_history_train= 2*(precision_history1*recall_history1)/(precision_history1+recall_history1)
f1_history_train[where(np.isnan(f1_history_train) == True)] = 0
f1_history_test=2*(precision_history2*recall_history2)/(precision_history2+recall_history2)
f1_history_test[where(np.isnan(f1_history_test) == True)] = 0

display_training_curves(f1_history_train, f1_history_test, 'f1_score', 211)
display_training_curves(history_1.history['AUC'], history_1.history['val_AUC'], 'auc', 212)

#Display confusion matrix
display_confusion_matrix(cmat, score, precision, recall)   ####2

#Display ROC
plot_roc('ROC Thermy tests', Y_test, new_model.predict(X_test))

#Getting max indez from f1_score 
index=where(f1_history_test==np.max(f1_history_test))
index=index[0][0]

#MAXIMUM metrics
print()
print()
print("MAXIMUM METRICS")
print()
print("train_acc:")
print(history_1.history['accuracy'][index])
print()
print("test_acc:")
print(history_1.history['val_accuracy'][index])
print()
print("precision:")
print(history_1.history['val_precision'][index])
print()
print("recall")
print(history_1.history['val_recall'][index])
print()
print("f1_score")
print(f1_history_test[index])
print()
print("AUC")
print(history_1.history['val_AUC'][index])
print()


#AVERAGE METRICS
print()
print("AVERAGE METRICS")
print()
print("train_acc:")
print(np.sum(history_1.history['accuracy'])/len(history_1.history['accuracy']))
print()
print("test_acc:")
print(np.sum(history_1.history['val_accuracy'])/len(history_1.history['val_accuracy']))
print()
print("precision:")
print(np.sum(history_1.history['val_precision'])/len(history_1.history['val_precision']))
print()
print("recall")
print(np.sum(history_1.history['val_recall'])/len(history_1.history['val_recall']))
print()
print("f1_score")
print(np.sum(f1_history_test)/len(f1_history_test))
print()
print("AUC")
print(np.sum(history_1.history['val_AUC'])/len(history_1.history['val_AUC']))
print()


In [ ]:
print("hola")